In [1]:
import os
import random
from copy import deepcopy

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import torch
import torch_geometric

import convert
import network
from lammps_scripts import LJSimulationStandalone
from simulation import (
    CompressionSimulation,
    LJSimulation,
    TemperatureRange,
    gen_sim_data,
    run_lammps_calc,
)
from utils import (
    add_edges_LJ,
    draw_graph,
    draw_network,
    inject_noise,
    recalc_bond,
    recalc_bonds,
    randomize_LJ
)


In [8]:
path = "/home/sergey/work/simulator_data_gen/data/binary/test_step_0.002"
chunks = []
for chunk_name in os.listdir(path):
    chunk = torch.load(os.path.join(path, chunk_name), weights_only=False)
    chunks.append(chunk)

In [9]:
torch.save(chunks, "/home/sergey/work/gnn/NN_Simulator/data/test_dePablo_stepsize_0.002.pt")

In [3]:
calc_dir = "/home/sergey/work/simulator_data_gen/scaling" # work

for i in range(5):
    loc_dir = os.path.join(calc_dir, str(i))
    os.makedirs(loc_dir)
    lj_sim = LJSimulation(
        n_atoms=120,
        atom_sizes=[16, 14, 12, 10],
        n_atom_types=4,
        box_dim=[-300, 300, -300, 300, -1, 1],
        temperature_range=TemperatureRange(0.0005, 0.0001, 10.0),
        n_steps=30_000,
    )
    lj_sim.write_to_file(loc_dir)
    run_lammps_calc(loc_dir, input_file='lammps.in')


In [7]:
skip = 1
data = []
for i in sizes:
    sim = []
    dump_file = os.path.join(calc_dir, str(i), "dump.lammpstrj")

    with open(dump_file, "r", encoding="utf8") as f:
            content = f.readlines()

    timesteps: list[int] = []
    for index, line in enumerate(content):
        if "ITEM: TIMESTEP" in line:
            timesteps.append(index)

    for i in range(0, len(timesteps) - 1, skip):
        timestep_data = content[timesteps[i] : timesteps[i + 1]]
        atoms = [list(map(lambda x: float(x), line.split()[1:3])) for line in timestep_data[9:]]
        graph = torch_geometric.data.Data(x=torch.tensor(atoms))
        graph.box = network.Box(-30, 30, -30, 30, -0.1, 0.1)
        graph = add_edges_LJ(graph, r=2.0)
        graph.edge_attr = torch.ones_like(graph.edge_attr)
        sim.append(graph)
    data.append(sim)
    # datasets[edge_radius] = data

In [12]:
torch.save(data, "/home/sergey/work/simulator_data_gen/diff_size.pt")

In [6]:
for key, value in datasets.items():
    torch.save(value, f"/home/sergey/work/simulator_data_gen/LJ_data_r{key}.pt")

In [ ]:
from convert import parse_dump

ord_data = []
for i in sizes:
    loc_dir = os.path.join(calc_dir, str(i))
    ord_network = network.Network.from_data_file(
        os.path.join(loc_dir, f"network{i}.lmp"),
        include_default_masses=1e6,
    )
    sim = parse_dump(
        os.path.join(loc_dir, "dump.lammpstrj"),
        ord_network,
        node_features="coord"
    )

    ord_data.append(sim)

ord_data

In [56]:
torch.save(data, "/home/sergey/work/gnn/NN_Simulator/LJ_data.pt")

In [ ]:
def draw_network(
    net: network.Network,
    edges: bool = True,
    periodic_edges: bool = True,
    box: bool = False,
    node_color: str = "skyblue",
    standalone: bool = True,
    node_size: float = 20,
    figure_scale: float = 3,
):
    if standalone:
        plt.figure(figsize=(8, 8))
    G = nx.Graph()
    # Add nodes
    for atom in net.atoms:
        G.add_node(atom.atom_id)

    pos = {atom.atom_id: (float(atom.y), float(atom.x)) for atom in net.atoms}

    # Add edges
    if edges:
        edge_index = [(bond.atom1.atom_id, bond.atom2.atom_id) for bond in net.bonds]
        naive_edge_lengths = [bond.atom1.dist(bond.atom2) for bond in net.bonds]
        for edge, length in zip(edge_index, naive_edge_lengths):
            if periodic_edges:
                G.add_edge(edge[0], edge[1])
            else:
                if length < net.box.x // 2:
                    G.add_edge(edge[0], edge[1])

    if box:
        B = nx.Graph()
        box_corners = [
            (net.box.x1, net.box.y1),
            (net.box.x1, net.box.y2),
            (net.box.x2, net.box.y1),
            (net.box.x2, net.box.y2),
        ]
        box_edges = [
            [box_corners[0], box_corners[1]],
            [box_corners[1], box_corners[2]],
            [box_corners[2], box_corners[3]],
            [box_corners[3], box_corners[0]],
        ]

        for corner, edge in zip(box_corners, box_edges):
            B.add_node(corner)
            B.add_edge(edge[0], edge[1])
        b_pos = {}
        b_pos[box_corners[0]] = (net.box.x1, net.box.y2)
        b_pos[box_corners[1]] = (net.box.x2, net.box.y2)
        b_pos[box_corners[2]] = (net.box.x2, net.box.y1)
        b_pos[box_corners[3]] = (net.box.x1, net.box.y1)

        nx.draw_networkx(B, b_pos, with_labels=False, node_color="black", node_size=1)

    nx.draw_networkx(
        G,
        pos,
        with_labels=False,
        node_color=node_color,
        node_size=node_size,
        font_size=10,
    )
    # nx.draw_networkx(G, with_labels=False, node_color=node_color, node_size=node_size, font_size=10)
    if standalone:
        plt.show()


In [ ]:
example = network.Network.from_data_file("data/raw/noised_prund/140_4/network_data/2/network.lmp")
draw_network(example, periodic_edges=False)

In [ ]:
from utils import load_data

data = load_data('noisy_networks.pt')